In [89]:
import pandas as pd
import numpy as np
from dateutil.parser import parse

In [90]:
s3_inhab = pd.read_pickle("inhabilitaciones.pkl")
s1_posesion = pd.read_hdf("s1_fecha_toma_posesion.h5")

In [91]:
s1_posesion.head()

,nombre_declaracion,fechaTomaPosesion
0,adrian hernandez hernandez,2011-03-01 00:00:00
1,abigain yareli hernandez cruz,2018-04-01 00:00:00
2,adelfo hernandez nieto,1992-11-01 00:00:00
3,adrian emilio madrigal gonzalez,2019-11-16 00:00:00
4,abel lemus torres,2021-05-07 00:00:00


In [92]:
s3_inhab.head()

,id,expediente,sancion_nombre,sancion_tipoPersona,sancion_objetoSocial,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,tipo_persona
0,a56b34a6-57c7-42b9-bb60-e214a65db391,PRA/68/2021,ofimagen sociedad anonima de capital variable,M,NaN,NaT,NaT,particular
1,622799f073ed865c36ba7cd5,07/2018,pedro barrueta escobar,F,FISICA,NaT,NaT,particular
2,622799f173ed865c36ba7cda,13/2019,monica virginia martinez bringas,F,PERSONA FÍSICA,NaT,NaT,particular
3,622799f173ed865c36ba7cdf,37/2020,rodriguez venegas iris ivonne,F,SD,NaT,NaT,particular
4,6317b5dd055a2a7017d97424,01/2021,"proyectos y construcciones r&m, sociedad anon...",F,"LA CONSTRUCCIÓN, REMODELACIÓN, MANTENIMIENTO, ...",NaT,NaT,particular


In [93]:
df = s1_posesion.merge(s3_inhab, left_on="nombre_declaracion", right_on="sancion_nombre", how = "inner", suffixes = ("s1", "s3") )

In [94]:
df = df.drop_duplicates()

In [95]:
df = df.dropna(subset=["nombre_declaracion","fechaTomaPosesion", "inhabilitacion_fechaInicial", "inhabilitacion_fechaFinal"])

In [96]:
df['fechaTomaPosesion'] = df['fechaTomaPosesion'].apply(lambda x: pd.to_datetime(x, utc=True))
df['inhabilitacion_fechaInicial'] = df['inhabilitacion_fechaInicial'].apply(lambda x: pd.to_datetime(x, utc=True))
df['inhabilitacion_fechaFinal'] = df['inhabilitacion_fechaFinal'].apply(lambda x: pd.to_datetime(x, utc=True))

df['fechaTomaPosesion'] = df['fechaTomaPosesion'].dt.date
df['inhabilitacion_fechaInicial'] = df['inhabilitacion_fechaInicial'].dt.date
df['inhabilitacion_fechaFinal'] = df['inhabilitacion_fechaFinal'].dt.date

df["posesion_durante_inhabilitacion"] = df.fechaTomaPosesion.between(df.inhabilitacion_fechaInicial, df.inhabilitacion_fechaFinal)

In [97]:
df[["nombre_declaracion","fechaTomaPosesion", "inhabilitacion_fechaInicial", "inhabilitacion_fechaFinal", "posesion_durante_inhabilitacion"]]

,nombre_declaracion,fechaTomaPosesion,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,posesion_durante_inhabilitacion
0,miguel angel solis flores,2021-05-14,2014-11-25,2034-11-19,True
3,roberto estrada cota,2022-02-01,2012-01-10,2022-09-28,True
5,jose luis suarez flores,2016-02-16,2008-01-10,2028-09-25,True
6,marco antonio martinez gutierrez,2019-01-01,2013-09-25,2023-09-22,True
8,marco antonio martinez gutierrez,2019-06-16,2013-09-25,2023-09-22,True
...,...,...,...,...,...
608,cynthia paola camacho torres,2021-06-01,2022-06-06,2022-06-07,False
609,cynthia paola camacho torres,2021-06-01,2022-08-31,2022-08-31,False
612,cayetano morales valdivia,2021-05-17,2021-11-30,2021-12-01,False
622,juan carlos martinez palacios,2017-03-03,2022-12-20,2023-12-20,False


In [98]:
df.posesion_durante_inhabilitacion.sum()

98

In [99]:
mask = df.posesion_durante_inhabilitacion
df["posesion_durante_inhabilitacion"] = df["posesion_durante_inhabilitacion"].astype(int)

In [100]:
resultado = df[mask]

In [101]:
len(resultado.nombre_declaracion.unique())

63

In [102]:
resultado = resultado.sort_values(by = "nombre_declaracion")
resultado = resultado.reset_index(drop=True)
resultado = resultado.drop(columns=["sancion_tipoPersona", "sancion_objetoSocial"])

In [103]:
resultado.tipo_persona.value_counts()

tipo_persona
servidor_publico    98
Name: count, dtype: int64

In [104]:
columnas_orden = ['nombre_declaracion', 'id', 'expediente', 'fechaTomaPosesion', 
       'inhabilitacion_fechaInicial', 'inhabilitacion_fechaFinal', 'posesion_durante_inhabilitacion']
resultado = resultado[columnas_orden]

In [105]:
resultado

,nombre_declaracion,id,expediente,fechaTomaPosesion,inhabilitacion_fechaInicial,inhabilitacion_fechaFinal,posesion_durante_inhabilitacion
0,aracely morales reyes,630d3b3790bf7b1af7b62c7b,0591/2013,2022-06-17,2014-05-09,2024-01-09,1
1,arturo martinez vazquez,630d3b3790bf7b1af7b62c50,CG DGAJR DRS 0004/2012,2022-08-15,2014-08-12,2024-04-12,1
2,cesar garcia lopez,630d3b3790bf7b1af7b62b68,0145/2016,2021-07-01,2017-11-04,2027-08-04,1
3,cesar garcia lopez,630d3b3790bf7b1af7b62b68,0145/2016,2020-06-01,2017-11-04,2027-08-04,1
4,daniel anteliz magana,4877,01/2020 TOMO A,2022-11-01,2021-06-25,2028-06-25,1
...,...,...,...,...,...,...,...
93,raul montes sarmiento,630d3b3490bf7b1af7b622c7,13/2008,2021-01-01,2012-10-10,2021-08-25,1
94,raul montes sarmiento,630d3b3490bf7b1af7b622c7,13/2008,2020-06-01,2012-10-10,2021-08-25,1
95,roberto estrada cota,630d3b3490bf7b1af7b6226a,R/0041/2010,2022-02-01,2012-01-10,2022-09-28,1
96,rocio fuentes hernandez,630d3b3490bf7b1af7b62259,R.A. 092/2012,2018-09-16,2013-03-20,2023-03-17,1


In [106]:
resultado.to_excel("posesion_durante_inhabilitacion_IntegriDAT.xlsx", index=False)